# App Search Engine exporter to Elasticsearch

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/enterprise-search/app-search-engine-exporter.ipynb)

This notebook explains the steps of exporting an App Search engine together with its configurations in Elasticsearch. This is not meant to be an exhaustive example for all App Search features as those will vary based on your instance, but is meant to give a sense of how you can export, migrate, and enhance your application.

We will look at:

- [how to export synonyms](#export-app-search-synonyms-in-elasticsearch)
- [how to export curations](#export-app-search-curations-in-elasticsearch)
- [how to create a new index in Elasticsearch](#create-a-new-elasticsearch-index)
- [how to add sparse vector fields](#add-sparse_vector-fields-for-semantic-search-optional)
- [how to query the new Elasticsearch index](#query-the-new-elasticsearch-index)

## Setup

Let's start by making sure our Elasticsearch and Enterprise Search clients are installed. We'll also use `getpass` to ensure we can allow secure user inputs for our IDs and keys to access our Elasticsearch instance.


In [ ]:
# install packages
import sys

!{sys.executable} -m pip install -qU elasticsearch elastic-enterprise-search

# import modules
from getpass import getpass
from elastic_enterprise_search import AppSearch
from elasticsearch import Elasticsearch
import json

## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?onboarding_token=search&utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial. 

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment. 

You will also need your **API KEY** to access your deployment. You can [create a new API key](https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key) from the `Stack Management -> API keys` menu in Kibana. Be sure to copy or write down your key in a safe place once it is created it will be displayed only upon creation.


In [ ]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

elasticsearch = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)

## Connect to App Search

For this notebook we will need access to an App Search private key that can access the App Search engine we want to export.
We will be instantiating the Enterprise Search client with the provided credentials and then check that we are correctly authenticated to Enterprise Search by getting the App Search engine details.

You can find your App Search endpoint and your search private key from the `Credentials` menu inside your App Search instance in Kibana.

Also note here, we define our `ENGINE_NAME`. For this example, we are using the `national-parks-demo` sample engine that is available within App Search.

In [ ]:
APP_SEARCH_ENDPOINT = getpass("App Search endpoint: ")
APP_SEARCH_PRIVATE_KEY = getpass("App Search private key: ")

app_search = AppSearch(APP_SEARCH_ENDPOINT, bearer_auth=APP_SEARCH_PRIVATE_KEY)

# modify this with your own engine name
ENGINE_NAME = "national-parks-demo"

## Export App Search synonyms in Elasticsearch

To get started with our export, we will first export any [synonyms](https://www.elastic.co/guide/en/app-search/current/synonyms-guide.html) we have in our App Search engine. 

The resulting synonyms will be placed into a new [Elasticsearch synoynm set](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-with-synonyms.html) named the same as our App Search egnine and used in analyzers for our synonyms-filter type later on.



In [ ]:
elasticsearch.synonyms.put_synonym(id=ENGINE_NAME, synonyms_set=[])

for synonym_set in app_search.list_synonym_sets(engine_name=ENGINE_NAME).body[
    "results"
]:
    elasticsearch.synonyms.put_synonym_rule(
        set_id=ENGINE_NAME,
        rule_id=synonym_set["id"],
        synonyms=", ".join(synonym_set["synonyms"]),
    )

Let's take a quick look at the synonyms we've migrated. We'll do this via the `GET _synonyms` endpoint

In [ ]:
print(json.dumps(elasticsearch.synonyms.get_synonym(id=ENGINE_NAME).body, indent=2))

## Export App Search curations in Elasticsearch

Next, we will export any curations that may be in our App Search engine.

To export App Search curations we will use Elasticsearch [query rules](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-using-query-rules.html). The code below will create the necessary `query_rules` to achieve this. Note that there is a default soft limit of 100 curations for `query_rules` that can be configured up to a hard limit of 1,000.

NOTE: This example outputs query rules requiring `exact` matches, which are case-sensitive. If you need typo tolerance, consider using `fuzzy`. If you need different case values consider adding multiple values to your criteria. 

In [ ]:
query_rules = []

for curation in app_search.list_curations(engine_name=ENGINE_NAME).body["results"]:
    if (curation["promoted"]):
      query_rules.append(
          {
              "rule_id": curation["id"] + "-pinned",
              "type": "pinned",
              "criteria": [
                  {
                      "type": "exact",
                      "metadata": "user_query",
                      "values": curation["queries"],
                  }
              ],
              "actions": {"ids": curation["promoted"]},
          }
      )
    if(curation["hidden"]):
      query_rules.append(
          {
              "rule_id": curation["id"] + "-exclude",
              "type": "exclude",
              "criteria": [
                  {
                      "type": "exact",
                      "metadata": "user_query",
                      "values": curation["queries"],
                  }
              ],
              "actions": {"ids": curation["hidden"]},
          }
      )

elasticsearch.query_rules.put_ruleset(ruleset_id=ENGINE_NAME, rules=query_rules)

Let's take a quick look at the query rules we've migrated. We'll do this via the `GET _query_rules/ENGINE_NAME` endpoint. Note that curations with both pinned and hidden documents will be represented as two rules in the ruleset.

In [ ]:
print(json.dumps(elasticsearch.query_rules.get_ruleset(ruleset_id=ENGINE_NAME).body, indent=2))

## Create a new Elasticsearch index

We recommend reindexing your App Search engine data into a new Elasticsearch index instead of reusing the existing one. This allows you to update the index mapping to take advantage of modern features like semantic search and the newly created Elasticsearch synonym set.

App Search has the following data types:

- `text`
- `number`
- `date`
- `geolocation`
 
Each of these types is mapped to Elasticsearch field types.

We can take a closer look at how App Search field types are mapped to Elasticsearch fields, by using the [`GET mapping API`](https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-get-mapping.html).
For App Search engines, the associated Elasticsearch index name is `.ent-search-engine-documents-[ENGINE_NAME]`, e.g. `.ent-search-engine-documents-national-parks-demo` for the App Search sample engine `national-parks-demo`.
One thing to notice is how App Search uses [multi-fields](https://www.elastic.co/guide/en/elasticsearch/reference/current/multi-fields.html) in Elasticsearch that allow for quickly changing the field type in App Search without requiring reindexing by creating subfields for each type of supported field:

<details>
  <summary>An example schema can be found by clicking here</summary>

```json
"[APP_SEARCH_ENGINE_FIELD_NAME]": {
  "type": "text",
  "fields": {
    "date": {
      "type": "date",
      "format": "strict_date_time||strict_date",
      "ignore_malformed": true
    },
    "delimiter": {
      "type": "text",
      "index_options": "freqs",
      "analyzer": "iq_text_delimiter"
    },
    "enum": {
      "type": "keyword",
      "ignore_above": 2048
    },
    "float": {
      "type": "double",
      "ignore_malformed": true
    },
    "joined": {
      "type": "text",
      "index_options": "freqs",
      "analyzer": "i_text_bigram",
      "search_analyzer": "q_text_bigram"
    },
    "location": {
      "type": "geo_point",
      "ignore_malformed": true,
      "ignore_z_value": false
    },
    "prefix": {
      "type": "text",
      "index_options": "docs",
      "analyzer": "i_prefix",
      "search_analyzer": "q_prefix"
    },
    "stem": {
      "type": "text",
      "analyzer": "iq_text_stem"
    }
  },
  "index_options": "freqs",
  "analyzer": "iq_text_base"
}
```
</details>

In our case we can assume that we have a well established schema and we do not need to use all multi-fields.

We can retrieve the field types of an App Search engine using the [Schema API](https://www.elastic.co/guide/en/app-search/current/schema.html) and then construct our mapping.

Also note that below, we set up variables for our `SOURCE_INDEX` and `DEST_INDEX`. If you want your destination index to be named differently, you can edit it here as these variables are used throughout the rest of the notebook.

First, we'll start by defining our source and destination indices. We'll also ensure that if the destination index is deleted if it exists, so that we start fresh.

In [7]:
# define SOURCE_INDEX and DEST_INDEX which we will continue to reuse; feel free to adjust DEST_INDEX
SOURCE_INDEX = ".ent-search-engine-documents-" + ENGINE_NAME
DEST_INDEX = "new-" + ENGINE_NAME

# delete the index if it's already created
if elasticsearch.indices.exists(index=DEST_INDEX):
    elasticsearch.indices.delete(index=DEST_INDEX)

Next, we'll create our settings which includes filters and and analyzers to use for our text fields.

These are similar to the Elasticsearch analyzers we use for App Search. The main difference is that we are also adding a synonyms filter so that we can
leverage the Elasticsearch synonym set we created in a previous step. If you want a different mapping for text fields, feel free to modify.

To start with, we'll define a number of filters that we can reuse in our analyzer itself. These include:
* `front_ngram`: defines a front loaded [n-gram token filter](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-edgengram-tokenfilter.html) that can help create prefixes for terms.
* `bigram_max_size`: defines a [maximum length](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-length-tokenfilter.html) for any bigram. In our example, we exclude any bigrams larger than 16 characters.
* `en-stem-filter`: defines [a stemmer](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-stemmer-tokenfilter.html) for use with English text.
* `bigram_joiner_unigrams`: a filter that [adds word n-grams](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-shingle-tokenfilter.html) into our token stream. This helps to expand the query to capture more context.
* `delimiter`: a [word delimiter graph token filter](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-word-delimiter-graph-tokenfilter.html) with this rules we've set on how to explicitly split tokens in our input.
* `en-stop-words-filter`: a default [stop token filter](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-stop-tokenfilter.html) to remove common English terms from our input.
* `synonyms-filter`: a [synonym graph token filter](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-synonym-graph-tokenfilter.html) that allows us to reuse the synonym set that we've defined above.


In [8]:
settings_analysis_filters = {
    "front_ngram": {"type": "edge_ngram", "min_gram": "1", "max_gram": "12"},
    "bigram_joiner": {
        "max_shingle_size": "2",
        "token_separator": "",
        "output_unigrams": "false",
        "type": "shingle",
    },
    "bigram_max_size": {"type": "length", "max": "16", "min": "0"},
    "en-stem-filter": {"name": "light_english", "type": "stemmer"},
    "bigram_joiner_unigrams": {
        "max_shingle_size": "2",
        "token_separator": "",
        "output_unigrams": "true",
        "type": "shingle",
    },
    "delimiter": {
        "split_on_numerics": "true",
        "generate_word_parts": "true",
        "preserve_original": "false",
        "catenate_words": "true",
        "generate_number_parts": "true",
        "catenate_all": "true",
        "split_on_case_change": "true",
        "type": "word_delimiter_graph",
        "catenate_numbers": "true",
        "stem_english_possessive": "true",
    },
    "en-stop-words-filter": {"type": "stop", "stopwords": "_english_"},
    "synonyms-filter": {
        "type": "synonym_graph",
        "synonyms_set": ENGINE_NAME,
        "updateable": True,
    },
}

Now we'll create our analyzer that utilizes these filters. The various parts of the analyzer will be used in different parts of our field mapping for text, and will help us to be able to index and query our text in different ways. These include:

* `iq_text_delimiter` is used for tokenizing and searching terms split on our specified delimiters in our text.
* `i_prefix` and `q_prefix` define our indexing and query tokenizers for creating prefix versions of our terms.
* `iq_text_stem` is used to create and query on stemmed versions of our tokens.
* `i_text_bigram` and `q_text_bigram` define our tokenizers for indexing and querying to create bigram terms.
* `i_text_base` and `q_text_base` define the indexing and query tokenization rules for general text tokenization.

In [9]:
settings_analyzer = {
    "i_prefix": {
        "filter": ["cjk_width", "lowercase", "asciifolding", "front_ngram"],
        "tokenizer": "standard",
    },
    "iq_text_delimiter": {
        "filter": [
            "delimiter",
            "cjk_width",
            "lowercase",
            "asciifolding",
            "en-stop-words-filter",
            "en-stem-filter",
        ],
        "tokenizer": "whitespace",
    },
    "q_prefix": {
        "filter": ["cjk_width", "lowercase", "asciifolding"],
        "tokenizer": "standard",
    },
    "i_text_base": {
        "filter": [
            "cjk_width",
            "lowercase",
            "asciifolding",
            "en-stop-words-filter",
        ],
        "tokenizer": "standard",
    },
    "q_text_base": {
        "filter": [
            "cjk_width",
            "lowercase",
            "asciifolding",
            "en-stop-words-filter",
            "synonyms-filter",
        ],
        "tokenizer": "standard",
    },
    "iq_text_stem": {
        "filter": [
            "cjk_width",
            "lowercase",
            "asciifolding",
            "en-stop-words-filter",
            "en-stem-filter",
        ],
        "tokenizer": "standard",
    },
    "i_text_bigram": {
        "filter": [
            "cjk_width",
            "lowercase",
            "asciifolding",
            "en-stem-filter",
            "bigram_joiner",
            "bigram_max_size",
        ],
        "tokenizer": "standard",
    },
    "q_text_bigram": {
        "filter": [
            "cjk_width",
            "lowercase",
            "asciifolding",
            "synonyms-filter",
            "en-stem-filter",
            "bigram_joiner_unigrams",
            "bigram_max_size",
        ],
        "tokenizer": "standard",
    },
}

Finally, we'll combine our filters and our analyzer into a settings object that we can use to define our destination index's settings.

More information on creating custom analyzers can be found in the [Elasticsearch documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-custom-analyzer.html).

In [10]:
settings = {
    "analysis": {
        "filter": settings_analysis_filters,
        "analyzer": settings_analyzer,
    }
}

Now that we have our settings built for our analysis, we'll get the current schema from our App Search engine and use that to build the mappings for our destination index we will be migrating the data into.

For any text fields, we'll explicitly define that mappings for how we want these fields to be stored. We define a number of fields here to emulate what App Search does underneath the hood. These include:
* A `keyword` field that ignores any token greater than 2048 characters in length.
* A `delimiter` field that captures any delimiters that we've defined in the above `delimiter` analysis.
* A `joined` field that uses our bigram analysis from above. This will create pairs of joined tokens that can be used for phrase queries.
* A `prefix` field that uses our prefix analysis from above. This is used for prefix wildcard to allow for partial matches as well as autocomplete queries.
* A `stem` field that captures the stemmed versions of our tokens.

Finally, the overall text field will be fully stored and analyzed using our base analyzer that we've defined above.

In [11]:
# get the App Search engine schema
schema = app_search.get_schema(engine_name=ENGINE_NAME)

# construct the Elasticsearch mapping
mapping = {}

for field_name in schema:
    field_type = schema[field_name]

    if field_type == "date":
        mapping[field_name] = {
            "type": "date",
            "format": "strict_date_time||strict_date",
            "ignore_malformed": True,
        }
    elif field_type == "location":
        mapping[field_name] = {"type": "geo_point", "ignore_z_value": False}
    elif field_type == "number":
        mapping[field_name] = {"type": "double"}
    elif field_type == "text":
        # feel free to modify this with your own mapping for text fields
        mapping[field_name] = {
            "fields": {
                "keyword": {"type": "keyword", "ignore_above": 2048},
                "delimiter": {
                    "type": "text",
                    "index_options": "freqs",
                    "analyzer": "iq_text_delimiter",
                },
                "joined": {
                    "type": "text",
                    "index_options": "freqs",
                    "analyzer": "i_text_bigram",
                    "search_analyzer": "q_text_bigram",
                },
                "prefix": {
                    "type": "text",
                    "index_options": "docs",
                    "analyzer": "i_prefix",
                    "search_analyzer": "q_prefix",
                },
                "stem": {"type": "text", "analyzer": "iq_text_stem"},
            },
            "type": "text",
            "index_options": "freqs",
            "analyzer": "i_text_base",
            "search_analyzer": "q_text_base",
        }

And now, we create our destination index that uses our mappings and analysis settings.

In [ ]:
# and actually create our index
elasticsearch.indices.create(
    index=DEST_INDEX, mappings={"properties": mapping}, settings=settings
)

# Add semantic text fields for semantic search (optional)

One of the advantages of exporting our index directly to Elasticsearch is that we can easily perform semantic search with ELSER. To do this, we'll need to add a `semantic_text` field to our index to use it. We will set up a `semantic_text` field using our default ELSER endpoint.

Note that to use this feature, your cluster must have at least one ML node set up with enough resources allocated to it.

If you do not have an ELSER endpoint running, it will be automatically downloaded, deployed and started for you when you use `semantic_text`. This means the first few commands may take a while as the model loads.

## Using semantic text fields for ingest and query

Next, we'll augment our text fields with `semantic_text` fields in our index. We'll do this by creating a `semtantic_text` field, and providing a `copy_to` directive from the original source field to copy the text into our semantic text fields.

In the example below, we are using the `description` and `title` fields from our example index to add semantic search on those fields.

In [ ]:
# by default we are adding a `semantic_text` field for the "description" and "title" fields in our schema
# feel free to modify this list to only include the fields that are relevant
SEMANTIC_TEXT_FIELDS = ["description", "title"]

# add the semantic_text field to our mapping for each field defined
for field_name in SEMANTIC_TEXT_FIELDS:
    semantic_field_name = field_name + "_semantic"
    mapping[semantic_field_name] = {
        "type": "semantic_text"
    }

# and for our text fields, add a "copy_to" directive to copy the text to the semantic_text field
for field_name in SEMANTIC_TEXT_FIELDS:
    semantic_field_name = field_name + "_semantic"
    mapping[field_name].update({"copy_to": semantic_field_name})

elasticsearch.indices.put_mapping(index=DEST_INDEX, properties=mapping)

## Reindex the data
Now that we have created the Elasticsearch index, it's time to reindex our data in the new index. If you are using the `semantic_text` fields as defined above with a `_semantic` suffix, and then the reindexing process will automatically infer the sparse vector values from ELSER and use those for the vectors as the reindex takes place.

In [15]:
reindex_task = elasticsearch.reindex(
    source={"index": SOURCE_INDEX},
    dest={"index": DEST_INDEX},
    wait_for_completion=False,
)

task_id = reindex_task["task"]

Note that above in the reindex command, we set `wait_for_completion` to false. Inference can possibly take a while and we might run the risk of our command timing out.
The call above will return a task that we can watch and see its progress the the `tasks` endpoint:

In [ ]:
task_response = elasticsearch.tasks.get(task_id=task_id)
print(json.dumps(task_response.body, indent=2))

# Query the new Elasticsearch index

We will exemplify:

- [how to replicate App Search queries](#how-to-build-app-search-like-queries)
- [how to do semantic search using ELSER](#how-to-do-semantic-search-using-elser)
- [how to combine App Search queries and ELSER](#how-to-combine-app-search-queries-with-elser)

### How to build App Search like queries

App Search exposes a [search_explain API](https://www.elastic.co/guide/en/app-search/current/search-explain.html) that receives an App Search query and returns the Elasticsearch query built by App Search.

```bash
curl -X POST '${ENTERPRISE_SEARCH_BASE_URL}/api/as/v1/engines/national-parks-demo/search_explain' \
-H 'Content-Type: application/json' \
-H 'Authorization: Bearer private-xxxxxx' \
-d '{
  "query": "park"
}'
```

From the output of the API call above, we can see the actual Elasticsearch query that will be used. Below, we are using this query as a base to build our own App Search like query using query rules and our Elasticsearch synonyms. The query is further enhanced by augmentation with the built-in App Search multifield types for such things as stemming and prefix matching.

To walk through a bit of what is happening in the query below. First, we gather some preliminary information about the fields we want to query and return.
1) We gather the fields we want for our result. This includes all the keys in the schema from above.
2) Next, we gather all of our text fields in our schema
3) And finally we gather the "best fields" which are those we want to query on using our stemmer.

In [19]:
QUERY_STRING = "park"

result_fields = list(schema.keys())

text_fields = [field_name for field_name in schema if schema[field_name] == "text"]
best_fields = [field_name + ".stem" for field_name in text_fields]

Now, from our text fields, we create set of fields with specified weights for our various analyzers.

* For the text field itself, we weight this as neutral, with a `1.0`
* For any stem fields, we weight this _slightly_ less to pull in closely stemmed words in the query.
* Any prefixes, we weight this with a minimal weight to ensure these do not dominate our scoring.
* For any potential bigram phrase matches, we weight these as well with a `0.75`
* Finally for our delimiter analyzed terms, we wight these somewhere in the middle.

These are the default weightings that App Search uses. Feel free to experiement with these values to find a balance that works for you.

In [20]:
cross_fields = []

for text_field in text_fields:
    cross_fields.append(text_field + "^1.0")
    cross_fields.append(text_field + ".stem^0.95")
    cross_fields.append(text_field + ".prefix^0.1")
    cross_fields.append(text_field + ".joined^0.75")
    cross_fields.append(text_field + ".delimiter^0.4")

Now we're ready to create our actual payload for our query. This is analagous to the query that App Search uses when querying.

Within this query, we first set an organic query rule. This defines a boolean query under the hood that allows a match to be found and scored either in our cross fields we defined above, or in the "best fields" as defined.

For the results, we sort on our score descending as the primary sort, with the document id as the secondary.

We apply highlights to returned text search descriptions, request a return size of the top 10 hits, and for each hit, return the result fields.

In [ ]:
app_search_query_payload = {
    "query": {
        "rule": {
            "organic": {
                "bool": {
                    "should": [
                        {
                            "multi_match": {
                                "query": QUERY_STRING,
                                "minimum_should_match": "1<-1 3<49%",
                                "type": "cross_fields",
                                "fields": cross_fields,
                            }
                        },
                        {
                            "multi_match": {
                                "query": QUERY_STRING,
                                "minimum_should_match": "1<-1 3<49%",
                                "type": "best_fields",
                                "fuzziness": "AUTO",
                                "prefix_length": 2,
                                "fields": best_fields,
                            }
                        },
                    ]
                }
            },
            "ruleset_ids": [ENGINE_NAME],
            "match_criteria": {"user_query": QUERY_STRING},
        }
    },
    "sort": [{"_score": "desc"}, {"_doc": "desc"}],
    "highlight": {
        "fragment_size": 300,
        "type": "plain",
        "number_of_fragments": 1,
        "order": "score",
        "encoder": "html",
        "require_field_match": False,
        "fields": { "description" : { "pre_tags" : ["<em>"], "post_tags" : ["</em>"] } },
    },
    "size": 10,
    "_source": result_fields,
}

print(f"Elasticsearch payload:\n{json.dumps(app_search_query_payload, indent=2)}")

Now that we have our fully flushed out query, we can use that to perform the actual search:

In [ ]:
results = elasticsearch.search(
    index=DEST_INDEX,
    query=app_search_query_payload["query"],
    highlight=app_search_query_payload["highlight"],
    source=app_search_query_payload["_source"],
    sort=app_search_query_payload["sort"],
    size=app_search_query_payload["size"],
)
print(json.dumps(results.body, indent=2))

### How to do semantic search using ELSER with semantic text fields

If you [enabled and reindexed your data with ELSER](#add-sparse_vector-fields-for-semantic-search-optional), we can now use this to do semantic search.
For each `semantic_text` field type, we can define a [match query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html) to easily perform a semantic search on these fields.

NOTE: For Elasticsearch versions prior to 8.18, we should update this to use a [semantic query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-semantic-query.html) to easily perform a semantic search on these fields.


In [ ]:
# replace with your own
QUERY_STRING = "best sunset view"
semantic_text_queries = []

for field_name in SEMANTIC_TEXT_FIELDS:
    semantic_field_name = field_name + "_semantic"
    semantic_text_queries.append(
        {
            "match": {
                semantic_field_name: QUERY_STRING
            }
        }
    )

semantic_query = {"bool": {"should": semantic_text_queries}}
print(f"Elasticsearch query:\n{json.dumps(semantic_query, indent=2)}\n")

In [ ]:
results = elasticsearch.search(index=DEST_INDEX, query=semantic_query, min_score=1)
print(f"Query results:\n{json.dumps(results.body, indent=2)}\n")

### How to combine App Search queries with semantic text

We will now provide an example on how to combine the previous two queries into a single query that applies both BM25 search and semantic search.
In the previous examples, we have a `bool` query with `should` clauses.
We will combine them in a single `bool` query and wrap this `bool` query in a `rule_query`.
The `rule_query` is used to pin results based on the query string, similarly to App Search curations.
The high-level structure of the query is following:

```json
GET [DEST-INDEX]
{
  "query": {
    "rule_query": {
      "organic": {
        "bool": {
          "should": [
            // multi_match query with best_fields from App Search generated query
            // multi_match query with cross_fields from App Search generated query
            // match queries for semantic_text fields
          ]
        }
      }  
    }
  }
}
```

We are again using `min_score` to exclude less relevant results.
In our example we are not boosting any of the `should` clauses, but this can be a way to boost ELSER results over BM25 results.

In [ ]:
payload = app_search_query_payload.copy()

for semantic_text_query in semantic_text_queries:
    payload["query"]["rule"]["organic"]["bool"]["should"].append(semantic_text_query)

print(f"Elasticsearch payload:\n{json.dumps(payload, indent=2)}\n")

In [ ]:
results = elasticsearch.search(
    index=DEST_INDEX,
    query=payload["query"],
    highlight=payload["highlight"],
    source=payload["_source"],
    sort=payload["sort"],
    size=payload["size"],
    min_score=1,
)

print(f"Semantic query results:\n{json.dumps(results.body, indent=2)}\n")